### Introduce the pre-trained model

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from jtk import RobertaJumanTokenizer

In [2]:
checkpoint = "nlp-waseda/roberta-base-japanese-with-auto-jumanpp"
# checkpoint = "cl-tohoku/bert-base-japanese-char-v2"
tokenizer = RobertaJumanTokenizer.from_pretrained("nlp-waseda/roberta-base-japanese")
model = AutoModelForMaskedLM.from_pretrained(checkpoint)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'RobertaJumanTokenizer'.


Exception: Can't find JUMAN command: jumanpp

In [8]:
tokenizer

PreTrainedTokenizerFast(name_or_path='nlp-waseda/roberta-base-japanese', vocab_size=32000, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=True)})

### Loading data

In [9]:
from datasets import load_dataset
dataset = load_dataset(path="dataset/processed/")

Using custom data configuration processed-56ae36322fe58fe1
Found cached dataset text (/home/tian/.cache/huggingface/datasets/text/processed-56ae36322fe58fe1/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad)


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 86555
    })
})

In [11]:
sample = dataset['train'].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Text: {row['text']}'")

Loading cached shuffled indices for dataset at /home/tian/.cache/huggingface/datasets/text/processed-56ae36322fe58fe1/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad/cache-64978b3cbd241a8a.arrow



'>>> Text: 許そうと思う夜道の沈丁花'

'>>> Text: 夏空で身長測定雲たちよ'

'>>> Text: しょうがつのかいものたてやまよくみえる'


In [12]:
def tokenize_function(examples):
    result = tokenizer(examples["text"], padding=True, return_tensors='pt')
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

In [13]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

Loading cached processed dataset at /home/tian/.cache/huggingface/datasets/text/processed-56ae36322fe58fe1/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad/cache-55d048f4f3b8c8d6.arrow


In [14]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 86555
    })
})

In [15]:
tokenized_datasets['train']['input_ids'][0], tokenized_datasets['train']['attention_mask'][0], tokenized_datasets['train']['word_ids'][0]

([2,
  10549,
  4778,
  8295,
  23367,
  23367,
  20762,
  1954,
  1652,
  1378,
  961,
  3008,
  769,
  368,
  1285,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [None,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None])

In [16]:
tokenizer.convert_tokens_to_ids(['UNK'])

[25624]

In [17]:
n = 0
for line in tokenized_datasets['train']['input_ids']:
    for w in line:
        if w == 25624:
            n += 1

print(f"#[UNK] = {n}")

#[UNK] = 0


In [18]:
tokenizer.decode(tokenized_datasets["train"][0]["input_ids"])

'[CLS] じゅうさんさいぱぱをおやじとよんでなつ[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

### Training model

In [19]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)

In [45]:
samples = [tokenized_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] じゅうさん[MASK]ぱぱ[MASK]おやじとよんでなつ[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

'>>> [CLS][MASK]るこにーでぶんこいちさつぶんのひや[MASK][SEP][PAD][PAD][PAD][PAD][PAD][PAD]'


In [50]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [51]:
model = AutoModelForMaskedLM.from_pretrained(checkpoint)

loading configuration file config.json from cache at /home/tian/.cache/huggingface/hub/models--nlp-waseda--roberta-base-japanese/snapshots/49ce73ecd6eb4dab3c1c06edaaab35892b3cea80/config.json
Model config RobertaConfig {
  "_name_or_path": "nlp-waseda/roberta-base-japanese",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 2,
  "classifier_dropout": null,
  "eos_token_id": 3,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file pytorch_model.bin from cache at /home/tian/.cache/huggingface/hub/m

In [52]:
training_args.num_train_epochs = 8.0
training_args.save_steps = 10000
training_args.weight_decay = 0.01
num_warmup_steps = 1_000,
init_lr = 2e-5,
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=False,
label_nam

In [53]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [54]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
/home/tian/anaconda3/envs/huggingface/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 86555
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 86560
  Number of trainable parameters = 110652416


  0%|          | 0/86560 [00:00<?, ?it/s]

{'loss': 4.6163, 'learning_rate': 4.971118299445471e-05, 'epoch': 0.05}
{'loss': 4.2999, 'learning_rate': 4.942236598890943e-05, 'epoch': 0.09}
{'loss': 4.1822, 'learning_rate': 4.9133548983364145e-05, 'epoch': 0.14}
{'loss': 4.1085, 'learning_rate': 4.884473197781886e-05, 'epoch': 0.18}
{'loss': 4.0244, 'learning_rate': 4.855591497227357e-05, 'epoch': 0.23}
{'loss': 3.9483, 'learning_rate': 4.826709796672828e-05, 'epoch': 0.28}
{'loss': 3.9225, 'learning_rate': 4.7978280961182996e-05, 'epoch': 0.32}
{'loss': 3.8385, 'learning_rate': 4.7689463955637706e-05, 'epoch': 0.37}
{'loss': 3.816, 'learning_rate': 4.740064695009242e-05, 'epoch': 0.42}
{'loss': 3.7875, 'learning_rate': 4.711182994454714e-05, 'epoch': 0.46}
{'loss': 3.7351, 'learning_rate': 4.682301293900185e-05, 'epoch': 0.51}
{'loss': 3.6842, 'learning_rate': 4.6534195933456565e-05, 'epoch': 0.55}
{'loss': 3.676, 'learning_rate': 4.6245378927911274e-05, 'epoch': 0.6}
{'loss': 3.577, 'learning_rate': 4.595656192236599e-05, 'epoch

Saving model checkpoint to test-trainer/checkpoint-10000
Configuration saved in test-trainer/checkpoint-10000/config.json


{'loss': 3.5315, 'learning_rate': 4.422365988909427e-05, 'epoch': 0.92}


...

Model weights saved in test-trainer/checkpoint-80000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-80000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-80000/special_tokens_map.json


{'loss': 2.2133, 'learning_rate': 3.5004621072088725e-06, 'epoch': 7.44}
{'loss': 2.1673, 'learning_rate': 3.211645101663586e-06, 'epoch': 7.49}
{'loss': 2.2702, 'learning_rate': 2.9228280961182996e-06, 'epoch': 7.53}
{'loss': 2.2308, 'learning_rate': 2.634011090573013e-06, 'epoch': 7.58}
{'loss': 2.1932, 'learning_rate': 2.3451940850277264e-06, 'epoch': 7.62}
{'loss': 2.2399, 'learning_rate': 2.05637707948244e-06, 'epoch': 7.67}
{'loss': 2.2039, 'learning_rate': 1.7675600739371536e-06, 'epoch': 7.72}
{'loss': 2.2261, 'learning_rate': 1.478743068391867e-06, 'epoch': 7.76}
{'loss': 2.2424, 'learning_rate': 1.1899260628465803e-06, 'epoch': 7.81}
{'loss': 2.2375, 'learning_rate': 9.011090573012939e-07, 'epoch': 7.86}
{'loss': 2.1981, 'learning_rate': 6.122920517560074e-07, 'epoch': 7.9}
{'loss': 2.226, 'learning_rate': 3.234750462107209e-07, 'epoch': 7.95}
{'loss': 2.211, 'learning_rate': 3.465804066543438e-08, 'epoch': 7.99}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 5274.7398, 'train_samples_per_second': 131.275, 'train_steps_per_second': 16.41, 'train_loss': 2.789232480019166, 'epoch': 8.0}


TrainOutput(global_step=86560, training_loss=2.789232480019166, metrics={'train_runtime': 5274.7398, 'train_samples_per_second': 131.275, 'train_steps_per_second': 16.41, 'train_loss': 2.789232480019166, 'epoch': 8.0})

In [55]:
trainer.save_model()

Saving model checkpoint to test-trainer
Configuration saved in test-trainer/config.json
Model weights saved in test-trainer/pytorch_model.bin
tokenizer config file saved in test-trainer/tokenizer_config.json
Special tokens file saved in test-trainer/special_tokens_map.json
